1) Importamos las librerias

In [1]:
import pandas as pd
import numpy as np
import json
import ast

1) Importamos los json

In [2]:
df_games = pd.read_json(r'Datasets\output_steam_games.json' ,lines=True)

In [3]:
data_list = []        # Lista para almacenar los diccionarios JSON de cada línea

file_path = r"Datasets\australian_user_reviews.json"

# Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

# Crear un DataFrame a partir de la lista de diccionarios
df_user_reviews = pd.DataFrame(data_list)

In [4]:
data_list = []   # Lista para almacenar los diccionarios JSON de cada línea

file_path = r"Datasets\australian_users_items.json"

# Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

df_user_items = pd.DataFrame(data_list)

#revisar que hace está funcion xd

In [5]:
print("la cantidad de filas de games son", df_games.shape[0])
print("la cantidad de filas de items son", df_user_items.shape[0])
print("la cantidad de filas de reviews son", df_user_reviews.shape[0])

la cantidad de filas de games son 120445
la cantidad de filas de items son 88310
la cantidad de filas de reviews son 25799


Borramos todas las filas que esten totalmente vacias

In [6]:
df_games = df_games.dropna(how ="all")
df_user_reviews = df_user_reviews.dropna(how ="all")
df_user_items = df_user_items.dropna(how ="all")

In [7]:
print("la cantidad de filas de games son", df_games.shape[0])
print("la cantidad de filas de items son", df_user_items.shape[0])
print("la cantidad de filas de reviews son", df_user_reviews.shape[0])
#vemos que había una enorme cantidad de filas vacias en df_games

la cantidad de filas de games son 32135
la cantidad de filas de items son 88310
la cantidad de filas de reviews son 25799


Desanidamos los datasets anidados

In [8]:
df_user_reviews_desc = df_user_reviews.explode(['reviews']) #descomponemos la columna reviews
df_reviews_series = df_user_reviews_desc['reviews'].apply(pd.Series) #creamos una serie a partir de ella
data_reviews = pd.concat([df_user_reviews_desc, df_reviews_series], axis=1) #creamos el dataframe definitivo 
                                                                            #y despues borramos la columna que quedó de más
data_reviews.drop('reviews', axis=1, inplace = True)

In [9]:
df_user_items_desc = df_user_items.explode(["items"])

In [26]:

df_user_items_desc2 = pd.json_normalize(df_user_items_desc['items'])


KeyboardInterrupt: 